In [ ]:
import numpy as np
import glob
import os
from models.JetPointNet import PointNetSegmentation


def load_data_from_npz(npz_file):
    data = np.load(npz_file)
    feats = data['feats']  # Shape: (num_samples, 859, 6)
    frac_labels = data['frac_labels']  # Shape: (num_samples, 859)
    tot_labels = data['tot_labels']  # Shape: (num_samples, 859)
    tot_truth_e = data['tot_truth_e']  # Shape: (num_samples, 859) (This is the true total energy deposited by particles into this cell)
    return feats, frac_labels, tot_labels, tot_truth_e

# Setup
os.environ['CUDA_VISIBLE_DEVICES'] = ""  # Disable GPU
model_path = "saved_model/PointNetModel.keras"

TEST_DIR = '/data/mjovanovic/jets/processed_files/2000_events_w_fixed_hits/SavedNpz/test'

model = PointNetSegmentation(num_points=859, num_classes=1)
model.load_weights(model_path)

npz_files = glob.glob(os.path.join(TEST_DIR, '*.npz'))



In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def plot_energies(predicted_energies, true_energies, coords, types, sample_index):
    fig = plt.figure(figsize=(14, 7))
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')

    ax1.set_title('Predicted Classifications')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax2.set_title('Actual Classifications')
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')

    vmax = 1  # Fixed as we now use binary classification

    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=vmax))
    sm.set_array([])

    for i, t in enumerate(types):
        if t == -1:
            continue

        color = 'r' if predicted_energies[i] > 0.5 else 'b'
        marker = 'x' if true_energies[i] > 0.5 else 'o'
        
        ax1.scatter(coords[i, 0], coords[i, 1], coords[i, 2], color=color, marker=marker)
        ax2.scatter(coords[i, 0], coords[i, 1], coords[i, 2], color=color, marker=marker)

    plt.suptitle(f'Sample {sample_index}')
    cbar = fig.colorbar(sm, ax=[ax1, ax2], shrink=0.5, aspect=20)
    cbar.set_label('Classification (0 or 1)')

    plt.show()

# Process each file
for npz_file_idx, npz_file in enumerate(npz_files):
    if npz_file_idx > 1:  # Only process the first two files for demonstration
        break

    feats, frac_labels, tot_labels, tot_truth_e = load_data_from_npz(npz_file)
    segmentation_logits, energies = model.predict(feats)
    segmentation_logits = np.squeeze(segmentation_logits, axis=-1)

    for sample_idx in range(min(10, len(feats))):
        types = feats[sample_idx][:, -1]
        valid_indices = types != -1  # Exclude -1 types for plotting

        coords = feats[sample_idx][valid_indices, :3]
        types_for_plotting = types[valid_indices]

        print(segmentation_logits)
        predicted_classes = np.where(segmentation_logits[sample_idx] > np.mean(segmentation_logits[sample_idx][valid_indices]), 1, 0)
        true_classes = np.where(frac_labels[sample_idx] > 0.5, 1, 0)

        predicted_energies_eval = predicted_classes * (types == 0)
        true_energies_eval = true_classes * (types == 0)

        tot_predicted = np.sum(predicted_energies_eval)
        tot_true = np.sum(true_energies_eval)

        print("Predicted Count: ", tot_predicted)
        print("True Count: ", tot_true)
        print("Percentage Difference: ", 100 * np.abs(tot_predicted - tot_true) / max(tot_true, 1), "% ")

        # Plot with adjusted classes
        #plot_energies(predicted_classes[valid_indices], true_classes[valid_indices], coords, types_for_plotting, sample_idx)


13/13 [==============================] - 42s 3s/step
[[-1.5399628  -1.5731173  -1.6062773  ... -1.0024456  -1.0024456
  -1.0024456 ]
 [-1.6372011  -1.6770837  -1.718894   ... -1.0375365  -1.0375365
  -1.0375365 ]
 [-1.2596424  -1.2724326  -1.2869823  ... -1.170547   -1.170547
  -1.170547  ]
 ...
 [-1.4278919  -1.4610045  -1.4954504  ... -0.9893812  -0.9893812
  -0.98938155]
 [-1.4234929  -1.44886    -1.4754001  ... -0.9882724  -0.9882724
  -0.9882724 ]
 [-1.5385374  -1.559716   -1.578083   ... -1.1586633  -1.1586633
  -1.1586633 ]]


ValueError: operands could not be broadcast together with shapes (168,) (859,) 